In [1]:
import pandas as pd

In [4]:
repo_dir = """C:\\Users\\yjpar\\Documents\\GitHub\\RGTM\\Cleaned\\"""
df1 = pd.read_csv(repo_dir + "merged_manhattan_cleanup.csv")

C:\Users\yjpar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df1["StreetAdd"] = df1.StreetNum + " " + df1.StreetName.apply(lambda x : x.strip())

In [34]:
df1.groupby(["StreetAdd"])["SalePrice"].count().to_clipboard()

C:\Users\yjpar\Anaconda3\lib\site-packages\pandas\io\clipboards.py:127: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding='utf-8', **kwargs)


In [11]:
len(df1.StreetAdd.unique())

11585

In [12]:
final = pd.read_csv(repo_dir+"MODEL_salesdate.csv")

In [108]:
final["BOM_DATE"] = pd.to_datetime(final.SaleDate)
final.SaleDate = pd.to_datetime(final.SaleDate)

C:\Users\yjpar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\yjpar\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [109]:
final["BOM_DATE"] = final.BOM_DATE.apply(lambda x : pd.datetime(x.year, x.month, 1))

C:\Users\yjpar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [110]:
medianSale = pd.read_csv(repo_dir + "MedianSalesDate.csv")
medianSale["BOM_DATE"] = pd.to_datetime(medianSale.BomDate)

In [111]:
medianSale =medianSale.drop("BomDate", axis = 1)

In [112]:
final1 = final.merge(medianSale, on="BOM_DATE", how="left")

In [113]:
recordedSale = pd.read_csv(repo_dir + "RecordedSales.csv")
recordedSale["BOM_DATE"] = pd.to_datetime(recordedSale.BomDate)
recordedSale=recordedSale.drop("BomDate", axis=1)

In [114]:
final2 = final1.merge(recordedSale, on="BOM_DATE", how="left")

In [115]:
final2.to_csv(repo_dir + "MODEL_salesdate_w_medSale_saleVolume.csv")

In [116]:
final3 = final2.copy(deep=True)

In [117]:
final3 = final3.drop(["BOM_DATE", "SaleDate"], axis=1)

In [118]:
final3.to_csv(repo_dir + "MODEL_medPrice_saleVol.csv")

In [84]:
alldf = pd.read_csv(repo_dir + "FINAL_ALL.csv")

In [85]:
alldf.columns

Index(['Unnamed: 0', 'AptNum', 'Baths', 'Beds', 'Block', 'BuildingCategoryDes',
       'BuildingCategoryNum', 'BuildingClassAtPresent',
       'BuildingClassAtTimeOfSale', 'CommercialUnits', 'GeoCode',
       'GrossSquareFeet', 'LandSquareFeet', 'Lot', 'Neighborhood',
       'ResidentialUnits', 'SaleDate', 'SalePrice', 'StreetAddress',
       'StreetName', 'StreetNum', 'TaxClassAtPresent', 'TaxClassAtTimeOfSale',
       'TotalUnits', 'NumComplaints', 'Year', 'YearBuilt', 'ZipCode',
       'halfMileAvgPrice6mo', 'NumComplaints3moPointOneMile',
       'NumFelonies3moPointOneMile', 'min_dist_park', 'min_dist_subway'],
      dtype='object')

In [86]:
alldf = alldf[["AptNum","BuildingCategoryDes","SalePrice","SaleDate","ZipCode","YearBuilt"]]

In [87]:
alldf.SaleDate = pd.to_datetime(alldf.SaleDate)
alldf["BuildingCategory"] = alldf.BuildingCategoryDes.apply(lambda x : 
                                                            "CONDOS" if x[:3]=="CON" else ("COOPS" if x[:3]=="COO" else "OTHERS"))

In [88]:
alldf.to_clipboard()

In [91]:
final_all = pd.read_csv(repo_dir + "FINAL_ALL_WITH_SALESDATE.csv")
final_all.SaleDate = pd.to_datetime(final_all.SaleDate)
final_all.BuildingCategory = final_all.BuildingCategory.apply(lambda x : x.strip())

In [92]:
alldf = alldf[["AptNum","BuildingCategory","SalePrice","SaleDate","ZipCode","YearBuilt"]]
final4 = final_all.merge(alldf, on =["BuildingCategory", "SaleDate", "SalePrice","ZipCode", "YearBuilt"], how="left")

In [105]:
final5 = final4[~((final4["BuildingCategory"].isin(["CONDOS","COOPS"]))&(final4["AptNum"]=="House")|(final4["YearBuilt"]==0))]

In [55]:
final5 = final5[~()]

In [96]:
final4.shape

(12147, 14)

In [106]:
final5.drop_duplicates().shape

(8601, 14)

In [107]:
final = final5.drop_duplicates()